Попробуем пообучать разные модели для предсказания удовлетворённости полётом.

Получим датасет из файла


*Подключим библиотеки

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
RANDOM_STATE = 42

In [3]:
df = pd.read_pickle ("fix_data.pkl ")

In [4]:
df.head()

,Age,Customer Type,Type of Travel,Inflight wifi service,Ease of Online booking,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,satisfaction,Class_Eco,Class_Eco Plus
0,48.0,0,1,3.0,3.0,5.0,3.0,5.0,5.0,3.0,2.0,5.0,4.0,5.0,5.0,0,0.0,0.0
1,35.0,1,1,2.0,2.0,3.0,5.0,4.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,1,0.0,0.0
2,41.0,1,1,4.0,4.0,5.0,5.0,5.0,3.0,3.0,3.0,3.0,4.0,3.0,5.0,1,0.0,0.0
3,50.0,1,1,2.0,2.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0,3.0,5.0,4.0,1,0.0,0.0
4,49.0,1,1,3.0,3.0,4.0,5.0,4.0,3.0,3.0,4.0,3.0,3.0,3.0,5.0,1,0.0,0.0


**Начнём с логистической регрессии**

Разобъем данные на тренировочные и тестовые

In [5]:
y = df['satisfaction']

X = df.drop(columns='satisfaction')

In [6]:
y

0         0
1         1
2         1
3         1
4         1
         ..
129873    0
129874    0
129877    0
129878    1
129879    0
Name: satisfaction, Length: 103401, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train.shape, X_test.shape,y_train.shape,y_test.shape

((77550, 17), (25851, 17), (77550,), (25851,))

In [8]:
y_train

72295     0
119194    1
87361     1
96846     1
73578     1
         ..
7954      1
68827     1
96433     1
1083      0
19857     0
Name: satisfaction, Length: 77550, dtype: int64

# Импорт библиотек

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score
from sklearn.preprocessing import MinMaxScaler


Воспользуемся скалером

In [10]:
ss = MinMaxScaler()
ss.fit(X_train)

X_train = pd.DataFrame(ss.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(ss.transform(X_test), columns=X_test.columns)

X_train.head()

,Age,Customer Type,Type of Travel,Inflight wifi service,Ease of Online booking,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Class_Eco,Class_Eco Plus
0,0.54,1.0,0.0,0.6,0.6,1.0,0.6,0.8,1.0,1.0,1.0,0.8,1.0,0.8,1.0,1.0,0.0
1,0.37,1.0,1.0,1.0,1.0,0.6,1.0,0.8,1.0,1.0,1.0,1.0,0.6,1.0,0.8,0.0,0.0
2,0.38,1.0,1.0,0.8,0.8,1.0,0.8,0.8,0.6,0.6,0.8,0.6,1.0,0.6,0.6,0.0,0.0
3,0.59,1.0,1.0,0.4,0.4,0.8,1.0,0.8,0.6,0.6,0.6,0.6,0.8,0.6,0.6,0.0,0.0
4,0.48,1.0,1.0,0.4,0.4,0.8,0.8,1.0,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.0,0.0


# Оценка важности признаков

In [16]:
X_train.shape

(77550, 17)

In [12]:
model = LogisticRegression()

model.fit(X_train, y_train)

pred = model.predict(X_test)

Веса модели

In [13]:
model.coef_, model.intercept_

(array([[-0.81962513,  1.93248066,  2.77753207,  1.90535799, -0.97941596,
         -0.14363531,  3.07742925,  0.31880205,  0.42550328,  1.49053974,
          1.23400534,  0.63845275,  1.54098492,  0.5772694 ,  1.06582482,
         -0.72737719, -0.83760371]]),
 array([-10.63093417]))

In [18]:
importances = pd.DataFrame({'weights': model.coef_[0], 'features': X_train.columns}).sort_values(by='weights')
importances.head(18)

,weights,features
4,-0.979416,Ease of Online booking
16,-0.837604,Class_Eco Plus
0,-0.819625,Age
15,-0.727377,Class_Eco
5,-0.143635,Food and drink
7,0.318802,Seat comfort
8,0.425503,Inflight entertainment
13,0.577269,Inflight service
11,0.638453,Baggage handling
14,1.065825,Cleanliness


In [20]:
accuracy_score(y_test, pred)


0.8738540095160728

In [21]:
recall_score(y_test, pred)


0.835703425453895

In [22]:
precision_score(y_test, pred)

0.8677563150074294